<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/Comparacion_Modelos_tecnicas_codificacion_categoricas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comparación de las Técnicas de Codificación de Variables Categóricas

En esta lección, vamos a comparar el desempeño de de las diferentes técnicas para codificar variables categóricas, utilizando dos modelos de machine learning: Random Forest (Bosque aleatorio) y Regresión Logística.

Vamos a comparar:

- Codificación One-hot 
- Reemplazar etiquetas por el número de observaciones
- Reemplazar categorías por números ordinales de acuerdo al target
- Codificación por la media
- WoE

y vamos a usar los datos del Titanic.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# carguemos el titanic dataset

# solo vamos a usar estar columnas en el demo
cols = ['pclass', 'age', 'sibsp', 'parch', 'fare',
        'sex', 'cabin', 'embarked', 'survived']

data = pd.read_csv('/content/drive/MyDrive/datasets/titanic.csv', usecols=cols)

data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B5,S
1,1,1,male,0.9167,1,2,151.5500,C22,S
2,1,0,female,2.0000,1,2,151.5500,C22,S
3,1,0,male,30.0000,1,2,151.5500,C22,S
4,1,0,female,25.0000,1,2,151.5500,C22,S


In [5]:
# revisemos si hay datos faltantes

data.isnull().sum()

pclass         0
survived       0
sex            0
age          263
sibsp          0
parch          0
fare           1
cabin       1014
embarked       2
dtype: int64

In [6]:
# Revisemos porcentaje de datos faltantes
data.isna().mean()

pclass      0.000000
survived    0.000000
sex         0.000000
age         0.200917
sibsp       0.000000
parch       0.000000
fare        0.000764
cabin       0.774637
embarked    0.001528
dtype: float64

In [7]:
# removamos las observaciones con datos ausentes NA en 
# las variables fare & embarked
data.dropna(subset=['fare', 'embarked'], inplace=True)

In [8]:
# Ahora reemplazamos los valores de la variable cabin 
# con la primera letra de cada una de sus etiquetas. 
# De esta forma simplificamos los valores en la variable 
# y facilitamos el demo 

data['cabin'] = data['cabin'].astype(str).str[0]


data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked
0,1,1,female,29.0000,0,0,211.3375,B,S
1,1,1,male,0.9167,1,2,151.5500,C,S
2,1,0,female,2.0000,1,2,151.5500,C,S
3,1,0,male,30.0000,1,2,151.5500,C,S
4,1,0,female,25.0000,1,2,151.5500,C,S


In [9]:
# remover las observaciones donde cabin = T
# ya que solo son unas pocas

data = data[data['cabin'] != 'T']

In [10]:
# separemos en sets de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(
    data.drop(labels='survived', axis=1),  # predictores
    data['survived'],  # target
    test_size=0.3,  # porcentaje de observaciones en el set de prueba
    random_state=0)  # semilla asegurar reproducibilidad

X_train.shape, X_test.shape

((913, 8), (392, 8))

In [11]:
# Reemplacemos los valores nulos por la media en las 
# variables numéricas 

def impute_na(df, variable, value):
    df[variable].fillna(value, inplace=True)

impute_na(X_test, 'age', X_train['age'].mean())
impute_na(X_train, 'age',  X_train['age'].mean())



en la celda anterior, primero reemplazamos (imputamos) el set de prueba con el valor promedio de la variable en el set de entrenamiento. De esta forma garantizamos que el valor promedio de la variable en el set de entrenamiento es el mismo cuando imputamos ambos sets.

In [12]:
X_train.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,female,30.00000,1,0,13.8583,n,C
698,3,male,18.00000,0,0,8.6625,n,S
1291,3,male,29.79847,0,0,8.7125,n,S
1229,3,male,27.00000,0,0,8.6625,n,S
118,1,male,29.79847,0,0,26.5500,D,S


In [13]:
# revisemos que no tenemos datos ausentes NA
# despues de la imputación
X_train.isnull().sum(), X_test.isnull().sum()

(pclass      0
 sex         0
 age         0
 sibsp       0
 parch       0
 fare        0
 cabin       0
 embarked    0
 dtype: int64, pclass      0
 sex         0
 age         0
 sibsp       0
 parch       0
 fare        0
 cabin       0
 embarked    0
 dtype: int64)

### Codificación One-Hot 

In [14]:
def get_OHE(df):

    df_OHE = pd.concat(
        [df[['pclass', 'age', 'sibsp', 'parch', 'fare']],
         pd.get_dummies(df[['sex', 'cabin', 'embarked']], drop_first=True)],
        axis=1)

    return df_OHE


X_train_OHE = get_OHE(X_train)
X_test_OHE = get_OHE(X_test)

X_train_OHE.head()

,pclass,age,sibsp,parch,fare,sex_male,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_n,embarked_Q,embarked_S
402,2,30.00000,1,0,13.8583,0,0,0,0,0,0,0,1,0,0
698,3,18.00000,0,0,8.6625,1,0,0,0,0,0,0,1,0,1
1291,3,29.79847,0,0,8.7125,1,0,0,0,0,0,0,1,0,1
1229,3,27.00000,0,0,8.6625,1,0,0,0,0,0,0,1,0,1
118,1,29.79847,0,0,26.5500,1,0,0,1,0,0,0,0,0,1


In [15]:
X_test_OHE.head()

,pclass,age,sibsp,parch,fare,sex_male,cabin_B,cabin_C,cabin_D,cabin_E,cabin_F,cabin_G,cabin_n,embarked_Q,embarked_S
586,2,29.00000,1,0,26.0000,0,0,0,0,0,0,0,1,0,1
200,1,46.00000,0,0,75.2417,1,0,1,0,0,0,0,0,0,0
831,3,40.00000,1,6,46.9000,1,0,0,0,0,0,0,1,0,1
1149,3,29.79847,0,0,7.7208,0,0,0,0,0,0,0,1,1,0
393,2,25.00000,0,0,31.5000,1,0,0,0,0,0,0,1,0,1


### Codificación por número de observaciones o frecuencia

In [16]:
def categorical_to_counts(df_train, df_test):

    # copia temporal de los dataframes originales 
    df_train_temp = df_train.copy()
    df_test_temp = df_test.copy( )

    for col in ['sex', 'cabin', 'embarked']:

        # dicccionario con el mapeo de categoría a conteo de frecuencia
        counts_map = df_train_temp[col].value_counts().to_dict()

        # reemplazar las etiquetas por el conteo respectivo
        df_train_temp[col] = df_train_temp[col].map(counts_map)
        df_test_temp[col] = df_test_temp[col].map(counts_map)

    return df_train_temp, df_test_temp


X_train_count, X_test_count = categorical_to_counts(X_train, X_test)

X_train_count.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,326,30.00000,1,0,13.8583,702,184
698,3,587,18.00000,0,0,8.6625,702,647
1291,3,587,29.79847,0,0,8.7125,702,647
1229,3,587,27.00000,0,0,8.6625,702,647
118,1,587,29.79847,0,0,26.5500,33,647


### Codificación por Números Ordinales


In [17]:
def categories_to_ordered(df_train, df_test, y_train, y_test):

    # copia temporal de los dataframes originales 
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # ordenar las categorías de acuerdo al promedio del target
        ordered_labels = df_train_temp.groupby(
            [col])['survived'].mean().sort_values().index

        #  diccionario con el mapeo de categoría ordenadas a un número ordinal
        ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

        # reemplazar las etiquetas por el número ordinal
        df_train_temp[col] = df_train[col].map(ordinal_label)
        df_test_temp[col] = df_test[col].map(ordinal_label)

    # remover el target
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_ordered, X_test_ordered = categories_to_ordered(
    X_train, X_test, y_train, y_test)

X_train_ordered.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,1,30.00000,1,0,13.8583,0,2
698,3,0,18.00000,0,0,8.6625,0,0
1291,3,0,29.79847,0,0,8.7125,0,0
1229,3,0,27.00000,0,0,8.6625,0,0
118,1,0,29.79847,0,0,26.5500,5,0


### Codificación por la Media

In [18]:
def categories_to_mean(df_train, df_test, y_train, y_test):

    # copia temporal de los dataframes originales 
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # calcular el promedio del target por categoría
        ordered_labels = df_train_temp.groupby(
            [col])['survived'].mean().to_dict()

       # reemplazar las etiquetas por el promedio del target
        df_train_temp[col] = df_train[col].map(ordered_labels)
        df_test_temp[col] = df_test[col].map(ordered_labels)

    # remover el target
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_mean, X_test_mean = categories_to_mean(
    X_train, X_test, y_train, y_test)

X_train_mean.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,0.730061,30.00000,1,0,13.8583,0.292023,0.516304
698,3,0.173765,18.00000,0,0,8.6625,0.292023,0.332303
1291,3,0.173765,29.79847,0,0,8.7125,0.292023,0.332303
1229,3,0.173765,27.00000,0,0,8.6625,0.292023,0.332303
118,1,0.173765,29.79847,0,0,26.5500,0.696970,0.332303


### Razon de probabilidades

In [19]:
def categories_to_ratio(df_train, df_test, y_train, y_test):

    # copia temporal de los dataframes originales 
    df_train_temp = pd.concat([df_train, y_train], axis=1).copy()
    df_test_temp = pd.concat([df_test, y_test], axis=1).copy()

    for col in ['sex', 'cabin', 'embarked']:

        # crear df con las diferentes partes de la ecuación del WoE
        # prob survived = 1
        prob_df = pd.DataFrame(df_train_temp.groupby([col])['survived'].mean())

        # prob survived = 0
        prob_df['died'] = 1-prob_df.survived

        # calcular WoE
        prob_df['Ratio'] = np.log(prob_df.survived/prob_df.died)

        # capturar WoE en un diccionario
        woe = prob_df['Ratio'].to_dict()

        # reemplazar las etiquetas por WoE
        df_train_temp[col] = df_train[col].map(woe)
        df_test_temp[col] = df_test[col].map(woe)

    # remover el target
    df_train_temp.drop(['survived'], axis=1, inplace=True)
    df_test_temp.drop(['survived'], axis=1, inplace=True)

    return df_train_temp, df_test_temp


X_train_ratio, X_test_ratio = categories_to_ratio(X_train, X_test, y_train, y_test)

X_train_ratio.head()

,pclass,sex,age,sibsp,parch,fare,cabin,embarked
402,2,0.994934,30.00000,1,0,13.8583,-0.885580,0.065241
698,3,-1.559176,18.00000,0,0,8.6625,-0.885580,-0.697788
1291,3,-1.559176,29.79847,0,0,8.7125,-0.885580,-0.697788
1229,3,-1.559176,27.00000,0,0,8.6625,-0.885580,-0.697788
118,1,-1.559176,29.79847,0,0,26.5500,0.832909,-0.697788


### Desempeño Random Forest

In [20]:
# crear una función para construir modelo Random Forest
# y comparar desempeño en los sets de entrenamiento y prueba


def run_randomForests(X_train, X_test, y_train, y_test):

    rf = RandomForestClassifier(n_estimators=50, random_state=39, max_depth=3)
    rf.fit(X_train, y_train)

    print('Set entrenamiento')
    pred = rf.predict_proba(X_train)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Set prueba')
    pred = rf.predict_proba(X_test)
    print(
        'Random Forests roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [21]:
# OHE
run_randomForests(X_train_OHE, X_test_OHE, y_train, y_test)

Set entrenamiento
Random Forests roc-auc: 0.8488938507340109
Set prueba
Random Forests roc-auc: 0.8072730715135779


In [22]:
# Conteos
run_randomForests(X_train_count, X_test_count, y_train, y_test)

Set entrenamiento
Random Forests roc-auc: 0.8654552920644698
Set prueba
Random Forests roc-auc: 0.8194309206967434


In [23]:
# Etiquetas ordenadas
run_randomForests(X_train_ordered, X_test_ordered, y_train, y_test)

Set entrenamiento
Random Forests roc-auc: 0.8669027820552304
Set prueba
Random Forests roc-auc: 0.8219733852645245


In [24]:
# Codificación por la media
run_randomForests(X_train_mean, X_test_mean, y_train, y_test)

Set entrenamiento
Random Forests roc-auc: 0.867010573863053
Set prueba
Random Forests roc-auc: 0.8207562479714378


In [25]:
# Razon de probabilidades
run_randomForests(X_train_ratio, X_test_ratio, y_train, y_test)

Set entrenamiento
Random Forests roc-auc: 0.867010573863053
Set prueba
Random Forests roc-auc: 0.8207562479714378


Al comparar los valores de roc_auc values en los sets de prueba, podemos ver que la codificación One-Hot tiene el peor desempeño.   Esto es de esperarse, si recordamos en como funcionan los árboles de decisión. Con la codificación One Hot creamos una variable dummy por cada categoría. Durante el proceso de entrenamiento, el árbol cada vez que se ramifica, considera cada una de las variables dummy como independiente. Cuando  tenemos una variable con alta cardinalidad, será poco probable que el algoritmo seleccione una de las variables dummy, ya que estas variables dummys son 'sparse' es decir, la mayoría de sus valores son cero y por lo tanto su poder predictivo es menor.

Por el contrario, el resto de la codificaciones tienen un mejor desempeño ya que retienen la relación entre la categoría y el target, lo cual incrementa su poder predictivo.  Sin embargo, ya que los algoritmos basados en árboles son modelos no-linales, las codificaciones basadas en la media del target ( o en relaciones monotónicas) no necesariamente mejorar su desempeño.

### Desempeño Regresión Logística

In [26]:
def run_logistic(X_train, X_test, y_train, y_test):

    # función para entrenar y evaluar desempeño de Regresión Logística
    logit = LogisticRegression(random_state=44, C=0.01)
    logit.fit(X_train, y_train)

    print('Set Entrenamiento')
    pred = logit.predict_proba(X_train)
    print(
        'Logistic Regression roc-auc: {}'.format(roc_auc_score(y_train, pred[:, 1])))

    print('Set Prueba')
    pred = logit.predict_proba(X_test)
    print(
        'Logistic Regression roc-auc: {}'.format(roc_auc_score(y_test, pred[:, 1])))

In [28]:
# OHE
run_logistic(X_train_OHE, X_test_OHE, y_train, y_test)

Set Entrenamiento
Logistic Regression roc-auc: 0.8287932450467097
Set Prueba
Logistic Regression roc-auc: 0.8013902412636589


In [29]:
# conteos
run_logistic(X_train_count, X_test_count, y_train, y_test)

Set Entrenamiento
Logistic Regression roc-auc: 0.7890283338466275
Set Prueba
Logistic Regression roc-auc: 0.7366114897760467


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [30]:
# Etiquetas ordenadas
run_logistic(X_train_ordered, X_test_ordered, y_train, y_test)

Set Entrenamiento
Logistic Regression roc-auc: 0.8223924648393389
Set Prueba
Logistic Regression roc-auc: 0.8006870063832089


In [31]:
# Codificación por la media
run_logistic(X_train_mean, X_test_mean, y_train, y_test)

Set Entrenamiento
Logistic Regression roc-auc: 0.7791217534134072
Set Prueba
Logistic Regression roc-auc: 0.7481878178080709


In [32]:
# Razon de probabilidades
run_logistic(X_train_ratio, X_test_ratio, y_train, y_test)

Set Entrenamiento
Logistic Regression roc-auc: 0.8508546350477364
Set Prueba
Logistic Regression roc-auc: 0.8204857730174184


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Para la Regresión Logística, el mejor desempeño se obtiene con la codificación One-Hot, ya que preserva la relación lineal entre las variables y el target y seguido por WoE y la codificación por números enteros ordenados.

Sin embargo, la codificación por conteo de frecuencia, tiene el peor desempeño ya que no crea una relación monotónica entre las variables y el target, y en este caso, la codificación promedio del target esta causando sobre-ajustes.